In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd

from sklearn.model_selection import train_test_split
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load Abalone dataset

In [3]:
# Load the Abalone dataset
# fetch dataset
abalone = fetch_ucirepo(id=1)

# data (as pandas dataframes)
X = abalone.data.features
y = abalone.data.targets
display(X)
display(y)

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550
...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960


,Rings
0,15
1,7
2,9
3,10
4,7
...,...
4172,11
4173,10
4174,9
4175,10


# Processing data

In [6]:
X.dtypes

Sex                object
Length            float64
Diameter          float64
Height            float64
Whole_weight      float64
Shucked_weight    float64
Viscera_weight    float64
Shell_weight      float64
dtype: object

In [201]:
def encode_data(df, encoder):
  df_encoded = df.copy()
  types, columns = df.dtypes, df.columns
  for i in range(len(types)):
    if types[i] == 'object':
      df_encoded[columns[i]] = encoder.fit_transform(df[columns[i]])
  return df_encoded

# Encode features
encoder = LabelEncoder()
X = encode_data(X, encoder)
y = encode_data(y, encoder)
X, y = X.values, y.values
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

display(X)
display(y)

array([[ 1.15198011, -0.57455813, -0.43214879, ..., -0.60768536,
        -0.72621157, -0.63821689],
       [ 1.15198011, -1.44898585, -1.439929  , ..., -1.17090984,
        -1.20522124, -1.21298732],
       [-1.28068972,  0.05003309,  0.12213032, ..., -0.4634999 ,
        -0.35668983, -0.20713907],
       ...,
       [ 1.15198011,  0.6329849 ,  0.67640943, ...,  0.74855917,
         0.97541324,  0.49695471],
       [-1.28068972,  0.84118198,  0.77718745, ...,  0.77334105,
         0.73362741,  0.41073914],
       [ 1.15198011,  1.54905203,  1.48263359, ...,  2.64099341,
         1.78744868,  1.84048058]])

array([[15],
       [ 7],
       [ 9],
       ...,
       [ 9],
       [10],
       [12]])

In [396]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y.reshape(-1, 1), test_size=0.2, random_state=42)

In [421]:
# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

print(X_train_tensor.shape)
print(X_test_tensor.shape)
print(y_train_tensor.shape)
print(y_test_tensor.shape)

torch.Size([3341, 8])
torch.Size([836, 8])
torch.Size([3341, 1])
torch.Size([836, 1])


# Build a neural network with Pytorch

In [402]:
class Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Init parameter for model

In [422]:
# Instantiate the model
input_size = X_train.shape[1]
output_size = len(torch.unique(y_train_tensor))
model = Model(input_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training the model
epochs = 500

In [394]:
# Print net architecture
print(model)

Model(
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=1, bias=True)
)


## Training model

In [423]:
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train_tensor)

    # Compute loss
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/500], Loss: 16.1634
Epoch [20/500], Loss: 7.1850
Epoch [30/500], Loss: 5.6360
Epoch [40/500], Loss: 5.2295
Epoch [50/500], Loss: 5.0495
Epoch [60/500], Loss: 4.9433
Epoch [70/500], Loss: 4.8702
Epoch [80/500], Loss: 4.8150
Epoch [90/500], Loss: 4.7706
Epoch [100/500], Loss: 4.7334
Epoch [110/500], Loss: 4.7018
Epoch [120/500], Loss: 4.6746
Epoch [130/500], Loss: 4.6509
Epoch [140/500], Loss: 4.6305
Epoch [150/500], Loss: 4.6123
Epoch [160/500], Loss: 4.5962
Epoch [170/500], Loss: 4.5816
Epoch [180/500], Loss: 4.5683
Epoch [190/500], Loss: 4.5560
Epoch [200/500], Loss: 4.5448
Epoch [210/500], Loss: 4.5346
Epoch [220/500], Loss: 4.5254
Epoch [230/500], Loss: 4.5166
Epoch [240/500], Loss: 4.5086
Epoch [250/500], Loss: 4.5009
Epoch [260/500], Loss: 4.4933
Epoch [270/500], Loss: 4.4860
Epoch [280/500], Loss: 4.4793
Epoch [290/500], Loss: 4.4730
Epoch [300/500], Loss: 4.4669
Epoch [310/500], Loss: 4.4611
Epoch [320/500], Loss: 4.4555
Epoch [330/500], Loss: 4.4498
Epoch [340/500], L

## Evalution model

In [425]:
# Evaluate the model on the test set
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test_tensor)
    # calculate error
    error = criterion(test_outputs, y_test_tensor)

    print(f'Test Mean Squared Error: {error:.4f}')

Test Mean Squared Error: 4.6378


In [426]:
rand_tensor = torch.rand(2,3)
print(rand_tensor)

tensor([[0.6229, 0.6095, 0.6635],
        [0.0152, 0.8841, 0.8759]])
